# Anomalies in Data

In [1]:
import pandas as pd
from siuba import *

import numpy as np

import ipywidgets as widgets
from ipywidgets import *
from IPython.display import Markdown
from IPython.core.display import display

In [2]:
df = pd.read_csv('gs://calitp-analytics-data/data-analyses/dla/e-76Obligated/obligated_list_1208.csv', low_memory=False)


/opt/conda/lib/python3.9/site-packages/google/auth/_default.py:73: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [3]:
df.sample()

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,...,project_location,type_of_work,seq,date_request_initiated,date_completed_request,mpo,warning,projectID,projectNO,compare_id_locode
13022,Obligated,ER,32L0(417),Humboldt County,2019-01-29,2019-01-30,2019-03-13,2019-03-15,2019-03-19,23849.98,...,"Mattole Road Pm 4.40, Humboldt County",Permanent Restoration,1,2019-01-29,2019-01-29,NON-MPO,NaN,32L0,417,False


In [4]:
df = df>>select(_.prefix,
           _.project_no,
           _.agency,
           _.prepared_date,
           _.fed_requested,
           _.ac_requested,
           _.total_requested,
           _.status_comment,
           _.locode, 
           _.dist,
           _.ftip_no,
           _.project_location, 
           _.type_of_work, 
           _.seq, 
           _.mpo)

In [5]:
df

,prefix,project_no,agency,prepared_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,ftip_no,project_location,type_of_work,seq,mpo
0,BPMPL,5904(121),Humboldt County,2018-12-18,0.00,0.0,0.00,Authorized,5904,1,HBPLOCAL,14 Bridges In Humboldt County,Bridge Preventive Maintenance - Deck Joints,3,NON-MPO
1,ER,32D0(008),Mendocino County,2018-12-17,11508.00,0.0,13000.00,Authorized,5910,1,NaN,"Comptche Ukiah Road, Cr 223 Pm 17.25",Permanent Restoration,3,NON-MPO
2,ER,4820(004),Humboldt County,2018-12-07,45499.64,0.0,51394.58,Authorized,5904,1,NaN,Mattole Rd Pm 43.17,Permanent Restoration,5,NON-MPO
3,CML,5924(244),Sacramento County,2018-12-11,207002.00,0.0,247002.00,Authorized,5924,3,SAC25086,Fair Oaks Blvd. Between Howe Ave And Munroe St,Create A Smart Growth Corridor With Barrier Se...,1,SACOG
4,CML,5924(214),Sacramento County,2018-12-05,0.00,5680921.0,5702041.00,Authorized,5924,3,SAC24753,Florin Rd Between Power Inn Rd. And Florin Per...,Streetscape (tc),3,SACOG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20112,FTACRRS,6000(069),San Francisco Bay Area Rapid Transit District,NaN,0.00,0.0,0.00,prog code,6000,4,NaN,FTA transfer,Bart Fare Collection Equipment,1,MTC
20113,FTASTPL,6343(006),Marin County Transit District,NaN,0.00,0.0,0.00,prog code,6343,4,NaN,FTA transfer,Bus Stops Improvement,1,MTC
20114,FTASTPL,6264(091),Santa Clara Valley Transportation Authority,NaN,0.00,0.0,0.00,prog code,6264,4,NaN,FTA transfer,Electronic Locker Upgrade And Replacement,1,MTC
20115,FTASTPL,6002(030),Alameda - Contra Costa Transit District,NaN,0.00,0.0,0.00,prog code,6002,4,NaN,FTA transfer,Quick Builds And Tempo Lane Delineation,1,MTC


## Looking into MPOs

using this map to see where the agency is in realtion to nearest mpo https://gisdata-caltrans.opendata.arcgis.com/datasets/b3e0ef95520843ba8c1d3b9c0fa9a607_0/explore

In [6]:
print(len(df>>count(_.mpo)>>arrange(-_.n)))

31


* Accoring to the Caltrans GIS data, there are 18 mpos

In [7]:
#number of obligations under MPO
df>>count(_.mpo)>>arrange(-_.n)

,mpo,n
18,SCAG,4988
12,MTC,3236
13,NON-MPO,2592
14,SACOG,1865
2,CFCG,1404
8,KCOG,998
26,STANCOG,759
0,AMBAG,745
21,SJCG,515
29,TCAG,489


### Which MPOs have the least amount of obligations? What Agencies do they include?

In [8]:
df >> count(_.mpo)>>arrange(-_.n)>> filter(_.n==1)

,mpo,n
6,FSTIP,1
11,MNOLTC,1
15,SANDA,1
20,SHASTA,1


#### MPO SANDA

In [9]:
df>>filter(_.mpo =='SANDA')

,prefix,project_no,agency,prepared_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,ftip_no,project_location,type_of_work,seq,mpo
12767,HSIPLN,5426(008),Big Bear Lake,2013-12-09,169650.0,0.0,188924.0,Authorized,NaN,8,SCAG015,Big Bear Lake Boulevard (sr 18) From Paine Roa...,Widen Shoulder For Two Way Left Turn Lane And ...,1,SANDA


* maybe meant "SANDAG" but Big Bear Lake is in "SCAG" jurisdiction
* looked to excel download file and this there is anothe Big Bear Lake and SANDAG instance

In [10]:
df>>filter(_.agency.str.contains('Big Bear')) >> filter(_.mpo.str.contains("SANDA"))

,prefix,project_no,agency,prepared_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,ftip_no,project_location,type_of_work,seq,mpo
12567,HSIPLN,5426(011),Big Bear Lake,2014-02-20,27000.0,0.0,30000.0,Authorized,NaN,8,SCAG015,"Big Bear Boulevard (sr 18) From 1,000 Ft. West...",Widen Shoulder For Two Way Left Turn Lane,1,SANDAG
12767,HSIPLN,5426(008),Big Bear Lake,2013-12-09,169650.0,0.0,188924.0,Authorized,NaN,8,SCAG015,Big Bear Lake Boulevard (sr 18) From Paine Roa...,Widen Shoulder For Two Way Left Turn Lane And ...,1,SANDA


google maps show the `project_location` intersections in the jurisdiction of Big Bear and not in SANDAG

#### MPO SHASTA

In [11]:
df >>filter(_.mpo=='SHASTA')

,prefix,project_no,agency,prepared_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,ftip_no,project_location,type_of_work,seq,mpo
10037,HSIP,5073(075),Orange,2014-12-30,196199.0,0.0,218000.0,Authorized,NaN,12,SCAG015,Intersection Of Tustin Street And Mayfair Avenue,Left-turn Signal Phasing,2,SHASTA


In [12]:
df>> filter(_.agency.str.contains('Shasta')) >> count(_.mpo)

,mpo,n
0,MTC,1
1,SACOG,3
2,STPA,129


In [13]:
df>>filter(_.agency.str.contains('Shasta')) >> filter(_.mpo=="MTC")

,prefix,project_no,agency,prepared_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,ftip_no,project_location,type_of_work,seq,mpo
19168,ACSTP,15A6(002),Shasta County,2021-05-18,0.0,19251.73,21746.0,Authorized,5921,2,NaN,Ca20-3 Zogg Fire Eo Repairs On Platina Road,Emergency Opening,1,MTC


* in the project location column, the intersection is Tustin and Mayfair which is in Orange Couny, or SCAG. Agency is also Orange, yet the mpo is Shasta

#### MPO MNOLTC and Agencies under

In [14]:
df>> filter(_.mpo=='MNOLTC')

,prefix,project_no,agency,prepared_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,ftip_no,project_location,type_of_work,seq,mpo
10820,RPSTP,5205(019),Rialto,2014-09-10,2000000.0,-2000000.0,0.0,Authorized,NaN,8,20040830,Former Pacific Electric Railroad Right-of-way ...,Extend Class 1 Bicycle Trail,2,MNOLTC


In [15]:
#looking to see if what other MPOs Rialto is in, should be in SCAG
df>> filter(_.agency=='Rialto') >> count(_.mpo)

,mpo,n
0,KCOG,1
1,MNOLTC,1
2,SANDAG,1
3,SCAG,15


In [16]:
df>> filter(_.agency=='Rialto') >> filter(_.mpo=='KCOG')

,prefix,project_no,agency,prepared_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,ftip_no,project_location,type_of_work,seq,mpo
10667,SRTS,5205(014),Rialto,2014-10-08,0.0,0.0,85287.07,Authorized,NaN,8,SCAG016,"Eucalyptus Avenue, King Street, 2nd Street, En...","Curb, Ramps, And In-pavement Warning Signals",3,KCOG


In [17]:
df>> filter(_.agency=='Rialto') >> filter(_.mpo=='SANDAG')

,prefix,project_no,agency,prepared_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,ftip_no,project_location,type_of_work,seq,mpo
12416,HSIP,5205(017),Rialto,2014-03-07,-89313.0,0.0,-99336.0,Authorized,NaN,8,SCAG015,Various Intersections Throughout The City Of R...,Pedestrian Countdown Heads At 67 Intersections,2,SANDAG


#### FSTIP

In [18]:
df>>filter(_.mpo=='FSTIP')

,prefix,project_no,agency,prepared_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,ftip_no,project_location,type_of_work,seq,mpo
12011,CML,5160(023),Los Banos,2014-04-16,-16940.31,0.0,-29901.84,Authorized,NaN,10,CMAQ11-05,In City Of Los Banos,Construct/install Pedestrian/bike Improvement,4,FSTIP


* FSTIP is the Federal Statewide Transportation Improvement Program. 

### Number of Unique Agencies in Each MPO

In [19]:

df >> group_by(_.mpo) >> summarize(n=_.agency.nunique()) >> arrange(-_.n) 

,mpo,n
18,SCAG,281
12,MTC,147
13,NON-MPO,132
26,STANCOG,89
22,SJCOG,71
16,SANDAG,70
9,MCAG,49
8,KCOG,44
14,SACOG,41
2,CFCG,24


In [20]:
df >> filter(_.mpo=="SCAG") >> count(_.agency) >> arrange(-_.n)

,agency,n
129,Los Angeles,478
130,Los Angeles County,419
195,Riverside County,104
261,Ventura County,102
204,San Bernardino County,94
...,...,...
256,Tustin,1
257,Twentynine Palm,1
259,Usda Forest Ser,1
272,Westminister,1


## Function for Agencies in each MPO

creating a function to look into each MPO. What agencies are listed under the MPO name? Are the projects located in that jurisdiction? 

In [21]:
@interact
def find_agencies(place=df.mpo.sort_values().unique().tolist()):
    mpos = df[df.mpo==place]
    display(Markdown(f"**Agencies in {place}**"))
    display(mpos >> count(_.agency) >> arrange(-_.n))
    pd.set_option('display.max_rows', 500)

interactive(children=(Dropdown(description='place', options=('AMBAG', 'BCAG', 'CFCG', 'COFCG', 'ER NONE', 'FCO…

In [22]:
#using Siuba filter, we can start to look at the agenies under different MPOs

In [23]:
df>>filter(_.agency=='Marin County')>> filter(_.mpo=='SCAG')

,prefix,project_no,agency,prepared_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,ftip_no,project_location,type_of_work,seq,mpo
19568,ACSTP,40A0(087),Marin County,2021-06-16,0.0,676463.01,764107.59,Authorized,5953,4,NaN,Lucas Valley Rd At Mp 3.92,Lucas Valley Rd At Mp 3.92 Roadway Damage. Un...,2,SCAG


* projet located in Lucas Valley, which is not in SCAG

In [24]:
df>>filter(_.agency=='Sonora')>> count(_.mpo)

,mpo,n
0,COFCG,1
1,NON-MPO,3
2,SCAG,2
3,STANCOG,1


In [25]:
df>>filter(_.agency=='Sonora')>> filter(_.mpo=='SCAG')

,prefix,project_no,agency,prepared_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,ftip_no,project_location,type_of_work,seq,mpo
8341,CML,5010(006),Sonora,2015-07-23,0.0,40296.0,0.0,Authorized,NaN,10,TRANS-HUB,S. Washington St. And Stockton St/hwy 49 (see ...,"Const Ped,transit Facility And Traffic Improv",2,SCAG
8637,CML,5010(008),Sonora,2015-07-08,205000.0,0.0,205000.0,Authorized,NaN,10,SONORA-PR,244 S. Shepherd St (apn 002-164-16)/nw Corner ...,"Parking Lot, Utilize As Park & Ride (tc)",1,SCAG


In [26]:
df>>filter(_.agency=='Shasta County')>> filter(_.mpo=='MTC')

,prefix,project_no,agency,prepared_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,ftip_no,project_location,type_of_work,seq,mpo
19168,ACSTP,15A6(002),Shasta County,2021-05-18,0.0,19251.73,21746.0,Authorized,5921,2,NaN,Ca20-3 Zogg Fire Eo Repairs On Platina Road,Emergency Opening,1,MTC


In [27]:
df>>filter(_.agency=='Marina')>> filter(_.mpo=='AMBAG')

,prefix,project_no,agency,prepared_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,ftip_no,project_location,type_of_work,seq,mpo
4828,RPSTP,5416(008),Marina,2017-02-28,-13679.6,0.0,-8226.94,Authorized,5416,5,BP01GP,Imjin Parkway From Reservation Road To Imjin Road,Construct Class Ii Bike Lanes (tc),7,AMBAG
5486,RPSTP,5416(008),Marina,2016-11-01,55689.0,0.0,55689.00,Authorized,5416,5,BP01GP,Imjin Parkway From Reservation Road To Imjin Road,Construct Class Ii Bike Lanes (tc),6,AMBAG
5557,RPSTP,5416(008),Marina,2016-07-27,-55689.0,0.0,-57389.00,Authorized,5416,5,BP01GP,Imjin Parkway From Reservation Road To Imjin Road,Construct Class Ii Bike Lanes (tc),5,AMBAG
6782,HSIP,5416(010),Marina,2016-05-05,1300033.0,0.0,1811770.00,Authorized,5416,5,HSIP08,Del Monte And Beach Road Roundabout,Construct Roundabout,2,AMBAG
9128,RPSTP,5416(011),Marina,2015-05-21,1650000.0,0.0,1650000.00,Authorized,5416,5,CM01MO,Imjin Road From 4-lane Section To Reservation ...,Widen Roadway From 2 To 4 Lanes (tc),1,AMBAG
9262,RPSTP,5416(008),Marina,2015-05-05,2000000.0,-2000000.0,0.00,Authorized,5416,5,BP01GP,Imjin Parkway From Reservation Road To Imjin Road,Construct Class Ii Bike Lanes (tc),4,AMBAG
12766,RPSTP,5416(008),Marina,2014-02-07,0.0,0.0,0.00,Authorized,5416,5,BP01GP,Imjin Parkway From Reservation Road To Imjin Road,Construct Class Ii Bike Lanes (tc),3,AMBAG
12824,RPSTP,5416(008),Marina,2014-01-09,0.0,2000000.0,2003300.00,Authorized,5416,5,BP01GP,Imjin Parkway From Reservation Road To Imjin Road,Construct Class Ii Bike Lanes (tc),2,AMBAG
14911,HSIP,5416(010),Marina,2019-10-08,-19.6,0.0,0.00,Authorized,5416,5,HSIP08,Del Monte And Beach Road Roundabout,Construct Roundabout,3,AMBAG


In [28]:
df>>filter(_.agency=='Kern County (District 9)')>> filter(_.mpo=='BCAG')

,prefix,project_no,agency,prepared_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,ftip_no,project_location,type_of_work,seq,mpo
2472,CML,5961(006),Kern County (District 9),2018-02-05,1327950.0,0.0,1500000.0,Authorized,NaN,9,KER160508,Backus Road From Tehachapi Willow Springs Road...,Surfacing Unpaved Shoulders,1,BCAG


In [29]:
df>>filter(_.agency=='Santa Barbara County')>> filter(_.mpo=='BCAG')

,prefix,project_no,agency,prepared_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,ftip_no,project_location,type_of_work,seq,mpo
293,ER,38F0(011),Santa Barbara County,2018-09-06,83500.0,0.0,83500.0,Authorized,5912,3,NaN,0.35 Miles West Of The Intersection Of Skyway ...,Remove Mbgr And Install Midwest Guardrail Syst...,3,BCAG
1041,ER,38F0(011),Santa Barbara County,2018-07-05,15000.0,-15000.0,0.0,Authorized,5912,3,NaN,0.35 Miles West Of The Intersection Of Skyway ...,Remove Mbgr And Install Midwest Guardrail Syst...,2,BCAG


In [30]:
df>>filter(_.agency=='Bishop')>> filter(_.mpo=='KCOG')

,prefix,project_no,agency,prepared_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,ftip_no,project_location,type_of_work,seq,mpo
10665,RPSTP,5125(021),Bishop,2014-09-23,4049.0,0.0,4575.0,Authorized,NaN,9,NaN,Intersection Of N. 3rd Street And E. Pine Stre...,"New 1,000 Foot Paved Bike Path",5,KCOG


In [31]:
df>>filter(_.agency=='Los Angeles')>> filter(_.mpo=='MCAG')

,prefix,project_no,agency,prepared_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,ftip_no,project_location,type_of_work,seq,mpo
10831,RPSTP,5953(685),Los Angeles,2014-09-05,378000.00,-378000.0,0.0,Authorized,NaN,7,LAF5514,Vermont Ave - From Manchester Blvd To El Segun...,Class Ii Bike Lane Construction(3 Miles),3,MCAG
10874,BRLSZD,5006(337),Los Angeles,2014-08-20,58000.00,0.0,72500.0,Authorized,NaN,7,LA000800,"Glendale-hyperion Viaduct, Bridge # 53c1882",Seismic Retrofit,8,MCAG
11404,HSIP,5006(551),Los Angeles,2014-07-01,0.00,0.0,-935.0,Authorized,NaN,7,SCAG015/HSIP2-,Roxford St And I-5 (the Golden State Freeway),Install New Traffic Signal,4,MCAG
11671,CML,5006(647),Los Angeles,2014-06-10,-0.01,0.0,0.0,Authorized,NaN,7,LAF1844,"City Of La, Crenshaw Blvd & Martin Luther King...",To Promote Pedestrian Activity In Project Limits,3,MCAG
11712,STPLZ,5953(564),Los Angeles,2014-04-28,767290.00,0.0,1039725.0,Authorized,NaN,7,LA000800,East Fork Road Over The North Fork Of San Gabr...,Seismic Retrofit,4,MCAG
11831,BRLSZD,5953(566),Los Angeles,2013-08-28,89563.00,0.0,402986.0,Authorized,NaN,7,LA000800,"Alameda Street Over Compton Creek, Bridge # 53...",Bridge Seismic Retrofit,3,MCAG


In [32]:
df>>filter(_.agency=='Los Angeles')>> count(_.mpo)

,mpo,n
0,ER NONE,1
1,KCOG,4
2,MCAG,6
3,NON-MPO,12
4,SANDAG,14
5,SCAG,478
6,SJCOG,4
7,STANCOG,3


In [33]:
df>>filter(_.agency=='Indio')>> filter(_.mpo=='MCTC')

,prefix,project_no,agency,prepared_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,ftip_no,project_location,type_of_work,seq,mpo
7984,CML,5275(022),Indio,2015-10-08,81189.0,0.0,109275.0,Authorized,NaN,8,RIV091208,Intersection Of Jackson Street And Market Stre...,Traffic Signals And Wireless Interconnect (tc),2,MCTC


In [34]:
df>>filter(_.agency=='Los Angeles')>> filter(_.mpo=='SJCOG')

,prefix,project_no,agency,prepared_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,ftip_no,project_location,type_of_work,seq,mpo
11012,RPSTP,5006(716),Los Angeles,2014-08-04,669000.00,-669000.0,0.0,Authorized,NaN,7,LAF3850/LA11G2,Vermont Ave B/n Hollywood Bl & Los Angeles Cit...,"Landscape, Install 6 Fully Irrigated Medians, Etc",3,SJCOG
11636,BHLS,5006(193),Los Angeles,2014-03-28,710400.00,0.0,888000.0,Authorized,NaN,7,LA000800,Soto St Bridge Over Valley Blvd & Uprr; Bridge...,Bridge Rehabilitation,9,SJCOG
11692,STP,5006(469),Los Angeles,2014-06-10,-39022.58,0.0,0.0,Authorized,NaN,7,NaN,"East Downtown, La (see State Comments Screen)",Truck Access Improvement,7,SJCOG
11801,HPLUL,5006(518),Los Angeles,2014-04-28,-127527.00,0.0,-159408.0,Authorized,NaN,7,LAE2147,Devonshire St And Woodley Ave,Lighting N/w San Fernando Valley,3,SJCOG


### MPOs with all correct agencies

* AMBAG
* KCAG
* SBCG
* TMPO

### MPOs with some misplaced agencies

* BCAG
* MTC
* SCAG
* KGOC
* MNOLTC
* FCOG 
* KCOG
* MCAG
* SJCOG
* STANCOG
* MCTC
* SACOG
* SJCOG
* STANCOG 
* TCAG


### MPOs not defined / all incorrect  
* CFCG - has Fresno Counties. Fresno is part of FCOG
* COFCG - combination of multiple MPOs
* FCOG - includes Ventura and Fresno
* FSTIP - a Prefix rather than a MPO
* MNOLTC - one entry
* SANDA - one entry
* SDAG - maybe SANDAG
* SANDAG - mostly SCAG Agencies
* SHASTA - one agency not in Shasta County
* SJCG - most likely SJCOG
* SLAC - maybe SLOCOG
* STACOG - STANCOG
* STNCOG - STANCOG
* STPA - maybe SRTA


## Agencies
Most of these agencies are corrected in the [change_agency_to_locodelist](https://github.com/cal-itp/data-analyses/blob/main/dla/e76_obligated_funds/change_agency_to_locodelist.ipynb)

In [35]:
df.agency.nunique()

661

In [36]:
sorted(df.agency.unique())

['Access Services',
 'Agoura Hills',
 'Alameda',
 'Alameda - Contra Costa Transit District',
 'Alameda Corridor Transportation Authority',
 'Alameda County',
 'Alameda County Congestion Management Agency',
 'Alameda County Transportation Commission',
 'Alameda County Transportation Improvement Authority',
 'Alameda County Waste Management Authority',
 'Albany',
 'Alhambra',
 'Alpine County',
 'Amador',
 'Amador City',
 'Amador County',
 'Amador County Transportation Commission',
 'Amador Rapid Transit System',
 'American Canyon',
 'Anaheim',
 'Angels',
 'Antelope Valley Transit Authority',
 'Antioch',
 'Apple Valley',
 'Arcadia',
 'Arcata',
 'Arroyo Grande',
 'Artesia',
 'Arvin',
 'Association Of Monterey Bay Area Governments',
 'Atascadero',
 'Atherton',
 'Atwater',
 'Auburn',
 'Avenal',
 'Azusa',
 'Bakersfield',
 'Baldwin Park',
 'Banning',
 'Barstow',
 'Bay Area Air Quality Management District',
 'Beaumont',
 'Bell',
 'Bell Gardens',
 'Bellflower',
 'Belmont',
 'Benicia',
 'Berkeley

In [75]:
df>>filter(_.agency.str.contains('Amador'))>>group_by(_.agency)>>count(_.project_no)

,agency,project_no,n
0,Amador,5926(031),2
1,Amador,5926(041),2
2,Amador,5926(049),3
3,Amador,5926(055),2
4,Amador City,5228(002),2
5,Amador County,32L0(147),3
6,Amador County,32L0(424),3
7,Amador County,5926(031),4
8,Amador County,5926(035),1
9,Amador County,5926(039),4


* list of agencies to be corrected with incorrect locodes
    * includes the null locodes

|Project No/Correct Locode |Agency in Obligated List | Actual Name (in Locode List) | 
| --------------- | --------------- | --------------- |
| 6053 | Sanbag  | San Bernardino Associated Governments | 
| 6066 | Sd Assoc Gov'T  | San Diego Association of Governments |
| 6071 | Ora Co Trans Au  | Orange County Transportation Authority |
| 6089 | Stan Area Aog  | Stanislaus Council of Governments |
| 5181 | Huntington Beac  | Huntington Beach |
| 6054 | Riv Co Trans Co | Riverside County Transportation Commission |
| 6128 |  Calaveras C T C | Calaveras Council of Governments |
| 5417 |  La Canada-Flint | La Canada Flintridge |
| 6303 | Sgvc | San Gabriel Valley Council of Governments |
| 6349 |  Stpd | Stockton Port District |
| 6305 |  Yrts | Yosemite Area Regional Transportation System JPA |
| 6248 |  Dwr | Department of Water Resources |
| 6308 |  Jpam | Transit Joint Powers Authority For Merced County |
| 6088 |  San Joaquin Cog  | San Joaquin Council of Governments |
| 5026 |  Sanbuenaventura | San Buenaventura |
| 5372 | Sanjuancapistra | San Juan Capistrano  |
| 5478 |  Rsm | Rancho Santa Margarita |
| 6190 |  Usda Forest Ser | U.S. Forest Service, Pacific Southwest Region |
| 6498 |  Cap Soeast Con  | Capital Southeast Connector |
| 6164 | Coachella V A G |  Coachella Valley Association of Governments |
| 6061 |  Fthll/East Tca | Foothill Eastern Transportation Corridor |
| 5476 |  Lgnw |  Laguna Woods |
| 6095 | Merced Co Aog | Merced County Association of Governments |
| 5413 |  Ranchopalosverd | Rancho Palos Verdes |
| 5349 |  Rollinghillsest | Rolling Hills Estates |
| 6373 |  Samd | South Coast Air Quality Management District |
| 6245 |  Sjrt |  San Joaquin Regional Transit District |
| 5926 | Amador | Amador County |
| 6128 | Calaveras C T C | Calaveras Council of Governments |
| 5930 | Calaveras | Calaveras County |
| 6498 | Cap Soeast Con | Capital Southeast Connector |
| 6164 | Coachella | Coachella Valley Association Of Governments |
| 5384 | Dst Hot Springs | Desert Hot Springs |
| 6061 | Fthll/East Tca | Foothill Eastern Transportation Corridor |
| 5230 | Gustine City | Gustine |
| 5387 | Hawaiian Garden | Hawaiian Gardens |
| 5958 | Imperial | Imperial County |
| 4825 | Imperial County | 5958 |
| 5948 | Inyo | Inyo County |
| 5940 | Mariposa | Mariposa County |
| 40A0 | Mendocino County | 5910 |
| 5939 | Merced | Merced County |
| 0484 | Merced County | Merced County |
| 5947 | Mono | Mono County |
| 22X0 | Portola Valley | 5390 |
| 5420 | Ranchocucamonga | Rancho Cucamonga |
| 5413 | Ranchopalosverd | Rancho Palos Verdes |
| 5956 | Riverside | Riverside County |
| 5349 | Rollinghillsest | Rolling Hills Estates |
| 5954 | San Bernardino | San Bernardino  County |
| 5957 | San Diego | San Diego County |
| 5929 | San Joaquin | San Joaquin County |
| 6088 | San Joaquin Cog | San Joaquin Council Of Governments |
| 5340 | Santa Fe Sprgs | Santa Fe Springs |
| 6066 | Sd Assoc Gov’T | San Diego Association of Governments |
| 20W0 | Sierra Madre | 5158 |
| 6503 | Solano County Transit | 6497 |
| 5938 | Stanislaus | Stanislaus County |
| 5932 | Tuolumne | Tuolumne County |
| 5448 | Twentynine Palm | Twentynine Palms |
| 5952 | Ventura | Ventura County |
| 5026 | San Buena Ventura | San Buenaventura |
| 5338 | Westminister | Westminster |

### Acronym Agencies Totals

In [38]:
#https://stackoverflow.com/questions/19937362/python-pandas-filter-string-data-based-on-its-string-length
df_acronym = df[df['agency'].apply(lambda x: len(x)<5)]

In [39]:
df_acronym>>count(_.agency, _.locode)

,agency,locode,n
0,Bell,5272,12
1,Brea,5237,11
2,Galt,5293,8
3,Ione,5311,2
4,Lodi,5154,14
5,Napa,5042,23
6,Ojai,5251,9
7,Ross,5176,14
8,Sgvc,7500,2
9,Taft,5193,40


In [40]:
df_acronym>>filter(_.agency=='Sgvc')>>count(_.project_no)

,project_no,n
0,6303(020),2
1,6303(026),1
2,6303(028),1
3,6303(030),1
4,7500(286),2


* only one acronym with a locode. checking agencies with no locode

In [41]:
df_acronym >> group_by(_.agency) >> summarize(n=_.project_no.nunique()) 

,agency,n
0,Bell,4
1,Brea,8
2,Dwr,3
3,Galt,3
4,Inyo,18
5,Ione,1
6,Jpam,3
7,Lgnw,1
8,Lodi,6
9,Mono,2


* 9 agencies that have acronyms when not including locodes

In [42]:
df_acronym>>filter(_.locode.isnull())>>count(_.agency)

,agency,n
0,Brea,5
1,Dwr,3
2,Inyo,24
3,Jpam,3
4,Lgnw,1
5,Lodi,2
6,Mono,4
7,Rsm,2
8,Samd,1
9,Sgvc,5


So the agency 'Sgvc' uses two 'locodes' in the `project_no` column, one of which '7500' we know is used with multiple agencies as seen in the [add_locodes](https://github.com/cal-itp/data-analyses/blob/main/dla/e76_obligated_funds/add_locodes.ipynb) notebook.

### **Agencies with Acronyms and NaN Locodes:**

In [43]:
print(len(df>>filter(_.locode.isnull())))

1499


In [44]:
# number of entries with no Locode- around ~7.5%

In [45]:
print(len(df>>filter(_.locode.isnull())>>count(_.agency)))

205


In [46]:
#number of agencies with Locodes missing around ~31% 

In [47]:
df>>filter(_.locode.isnull())>>count(_.agency)>>arrange(-_.n)

,agency,n
100,Los Angeles,97
154,San Joaquin,67
177,Stockton,59
107,Merced,51
176,Stanislaus,49
110,Modesto,44
150,San Diego,33
144,Riverside,29
186,Turlock,28
158,Sanbag,25



* using the most common first four numbers of the `project_no`


|Project No/Corresponding Locode |Agency in Obligated List | Actual Name (in Locode List) | 
| --------------- | --------------- | --------------- |
| 6053 | Sanbag  | San Bernardino Associated Governments | 
| 6066 | Sd Assoc Gov'T  | San Diego Association of Governments |
| 6071 | Ora Co Trans Au  | Orange County Transportation Authority |
| 6089 | Stan Area Aog  | Stanislaus Council of Governments |
| 5181 | Huntington Beac  | Huntington Beach |
| 6054 | Riv Co Trans Co | Riverside County Transportation Commission |
| 6128 |  Calaveras C T C | Calaveras Council of Governments |
| 5417 |  La Canada-Flint | La Canada Flintridge |
| 6303 | Sgvc | San Gabriel Valley Council of Governments |
| 6349 |  Stpd | Stockton Port District |
| 6305 |  Yrts | Yosemite Area Regional Transportation System JPA |
| 6248 |  Dwr | Department of Water Resources |
| 6308 |  Jpam | Transit Joint Powers Authority For Merced County |
| 6088 |  San Joaquin Cog  | San Joaquin Council of Governments |
| 5026 |  Sanbuenaventura | San Buenaventura |
| 5372 | Sanjuancapistra | San Juan Capistrano  |
| 5478 |  Rsm | Rancho Santa Margarita |
| 6190 |  Usda Forest Ser | U.S. Forest Service, Pacific Southwest Region |
| 6498 |  Cap Soeast Con  | Capital Southeast Connector |
| 6164 | Coachella V A G |  Coachella Valley Association of Governments |
| 6061 |  Fthll/East Tca | Foothill Eastern Transportation Corridor |
| 5476 |  Lgnw |  Laguna Woods |
| 6095 | Merced Co Aog | Merced County Association of Governments |
| 5413 |  Ranchopalosverd | Rancho Palos Verdes |
| 5349 |  Rollinghillsest | Rolling Hills Estates |
| 6373 |  Samd | South Coast Air Quality Management District |
| 6245 |  Sjrt |  San Joaquin Regional Transit District |


In [48]:
#printing number of entries for Westmorland which has an offical locode of 5278 
print(len(df>>filter(_.locode=="5278")))
print(len(df>>filter(_.agency=="Westmorland")))

11
12


In [49]:
df>>filter(_.agency=="Westmorland")>>count(_.agency)

,agency,n
0,Westmorland,12


In [50]:
df>>filter(_.agency.str.contains("Huntington Beac"))>>count(_.agency)

,agency,n
0,Huntington Beac,6
1,Huntington Beach,55


#### Dwr

In [51]:
df>>filter(_.agency=="Dwr")

,prefix,project_no,agency,prepared_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,ftip_no,project_location,type_of_work,seq,mpo
8953,BRLOZ,6248(011),Dwr,2015-06-12,-93034.07,0.0,-105087.58,Authorized,NaN,10,CT004,Mccabe Road Over California Aqueduct,Seismic Retrofit,2,SCAG
8954,BRLOZ,6248(013),Dwr,2015-06-12,-96147.13,0.0,-105365.27,Authorized,NaN,10,CT004,Butts Road Over California Aqueduct,Seismic Retrofit,2,SCAG
8955,BHLOZ,6248(030),Dwr,2015-06-12,-128952.96,0.0,-145660.16,Authorized,NaN,10,CT004,Mervel Avenue Over California Aqueduct,Seismic Retrofit,2,SCAG


* locode list shows this as "Department of Water Resources" 

#### Jpam

In [52]:
df>>filter(_.agency=="Jpam")

,prefix,project_no,agency,prepared_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,ftip_no,project_location,type_of_work,seq,mpo
9466,CML,6308(016),Jpam,2015-03-23,100000.00,0.0,100000.00,Authorized,NaN,10,TJPA02,Merced County Region,"Public Outreach And Marketing For ""the Bus"" (tc)",1,SCAG
10243,CML,6308(013),Jpam,2014-12-26,-183102.22,0.0,-183102.22,Authorized,NaN,10,TJPA02,Merced County Transit,Accuring Bus Wraps For Public Outreach(tc),2,SCAG
11751,CML,6308(014),Jpam,2014-05-20,100000.00,0.0,100000.00,Authorized,NaN,10,TJPA02,Merced County Region,"Public Outreach And Marketing For ""the Bus"" (tc)",1,STANCOG


* Showing in locode list as "Transit Joint Powers Authority For Merced County"

In [53]:
df>>filter(_.project_location=='Stanislaus County')>>count(_.agency)

,agency,n
0,Stan Area Aog,3
1,Stanislaus,2
2,Stanislaus Council Of Governments,4
3,Stanislaus County,3


In [54]:
df>>filter(_.project_location=='Stanislaus County')

,prefix,project_no,agency,prepared_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,ftip_no,project_location,type_of_work,seq,mpo
680,CML,6089(055),Stanislaus Council Of Governments,2018-08-28,0.00,0.0,0.00,Authorized,6089,10,ST07,Stanislaus County,Regional Rideshare Includes Public Outreach,2,STANCOG
5205,CML,6089(062),Stanislaus Council Of Governments,2016-12-22,-4000.02,0.0,0.00,Authorized,6089,10,ST07,Stanislaus County,Regional Rideshare Program,2,STANCOG
7330,CML,5938(217),Stanislaus County,2016-02-17,-2915.93,0.0,-7087.20,Authorized,5938,10,CQSC003,Stanislaus County,Replace Motor Grader,2,STANCOG
8464,CML,5938(232),Stanislaus,2015-08-04,71494.00,0.0,80757.00,Authorized,NaN,10,04STA048C,Stanislaus County,Rideshare Program,1,SCAG
9061,CML,5938(217),Stanislaus,2015-06-02,110000.00,0.0,267000.00,Authorized,NaN,10,CQSC003,Stanislaus County,Replace Motor Grader,1,SCAG
10242,STP,6089(060),Stan Area Aog,2014-12-30,0.00,0.0,0.00,Authorized,NaN,10,05STA002R,Stanislaus County,Planning And Monitoring Activities,2,SCAG
11689,CML,6089(062),Stan Area Aog,2014-06-04,168207.00,0.0,190000.00,Authorized,NaN,10,ST07,Stanislaus County,Regional Rideshare Program,1,STANCOG
12697,STP,6089(056),Stan Area Aog,2014-02-18,0.00,0.0,0.00,Authorized,NaN,10,05STA002R,Stanislaus County,Planning And Monitoring Activities,2,SANDAG
14059,STP,6089(079),Stanislaus Council Of Governments,2019-03-11,60000.00,0.0,67774.00,Authorized,6089,10,05STA002R,Stanislaus County,Planning And Monitoring Activities,1,STANCOG
16542,STPLNI,6089(079),Stanislaus Council Of Governments,2020-05-12,-0.01,0.0,-0.37,Authorized,6089,10,05STA002R,Stanislaus County,Planning And Monitoring Activities,3,STANCOG


* Agencies have no locodes listed under some obligations. 
* Agency name "Stanislaus" appears to be "Stanislaus County"

#### Agency Name 'YRTS'

In [55]:
df>>filter(_.agency=='Yrts')

,prefix,project_no,agency,prepared_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,ftip_no,project_location,type_of_work,seq,mpo


In [56]:
df>>filter(_.agency=='Yrts')>>count(_.locode)

,locode,n


In [57]:
df>>filter(_.agency=='Yrts')>>count(_.project_no)

,project_no,n


In [58]:
df>>filter(_.agency.str.contains('Yosemite'))>>count(_.locode)

,locode,n
0,6305,12


* 'Yrts' is actually ['Yarts'](https://yarts.com/), or Yosemite Area Regional Transportation System 

* no locodes listed, but if we use the first four numbers of the `project_no`, we get the same locode number as the `agency` Yosemite AND the official locode listed
* changing this in the "change_agency_to_locodelist" and uploading updated data in GCS

### How many agencies for a project location? 

In [59]:
#df>>group_by(_.project_location)>>count(_.agency)>>arrange(-_.n)>>filter(_.n>5)
df >> group_by(_.project_location) >> summarize(n=_.agency.nunique()) >> arrange(-_.n) >>filter(_.n>2)

,project_location,n
8196,Various Locations,12
2623,FTA transfer,9
1830,Citywide,7
8337,Various Locations Throughout The City,6
8079,Various,5
2083,Countywide,4
7558,Stanislaus County,4
4503,Interstate 5 From S/o Avenida Pico To S/o Aven...,3
5277,Merced County,3
6925,San Francisco Bay Area,3


In [60]:
df>>filter(_.project_location.str.contains("Interstate 5 From S/o Avenida Pico To S/o Aven"))

,prefix,project_no,agency,prepared_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,ftip_no,project_location,type_of_work,seq,mpo
4330,STPLN,6212(016),Caltrans,2017-04-11,0.0,0.0,0.00,Authorized,6212,12,ORA990929,Interstate 5 From S/o Avenida Pico To S/o Aven...,Add Hov Lane At Each Direction (tc),4,NON-MPO
4637,STPLN,6212(016),Caltrans,2016-12-05,1053000.0,0.0,-10922000.00,Authorized,6212,12,ORA990929,Interstate 5 From S/o Avenida Pico To S/o Aven...,Add Hov Lane At Each Direction (tc),3,NON-MPO
11160,CMLN,6071(071),Ora Co Trans Au,2014-07-30,0.0,0.0,0.00,Authorized,NaN,12,ORA990929,Interstate 5 From S/o Avenida Pico To S/o Aven...,Add Hov Lane At Each Direction,3,MCAG
16712,CMLN,6071(071),Orange County Transportation Authority,2020-06-25,100450000.0,0.0,-2209853.49,Authorized,6071,12,ORA990929,Interstate 5 From S/o Avenida Pico To S/o Aven...,Add Hov Lane At Each Direction,4,SCAG


In [61]:
df>>filter(_.project_location.str.contains("Merced County"))>>group_by(_.agency)>>count(_.project_no, _.locode)

,agency,project_no,locode,n
0,Merced County,5939(093),5939,1
1,Merced County Association Of Governments,6095(061),6095,1
2,Merced County Association Of Governments,6095(074),6095,2
3,Merced County Association Of Governments,6095(077),6095,1
4,Merced County Association Of Governments,6095(081),6095,2
5,San Joaquin Council Of Governments,6088(051),6088,3
6,San Joaquin Council Of Governments,6088(060),6088,4
7,Transit Joint Powers Authority For Merced County,6308(014),6308,1
8,Transit Joint Powers Authority For Merced County,6308(016),6308,1
9,Transit Joint Powers Authority For Merced County,6308(018),6308,3


In [62]:
df>>filter(_.project_location.str.contains("San Francisco Bay Area"))>>group_by(_.agency)>>count(_.project_no)

,agency,project_no,n
0,Bay Area Air Quality Management District,6297(007),2
1,Bay Area Air Quality Management District,6297(008),3
2,Bay Area Air Quality Management District,6297(009),3
3,Metropolitan Transportation Commission,6084(137),1
4,Metropolitan Transportation Commission,6084(146),2
5,Metropolitan Transportation Commission,6084(147),1
6,Metropolitan Transportation Commission,6084(151),1
7,Metropolitan Transportation Commission,6084(152),1
8,Metropolitan Transportation Commission,6084(155),1
9,Metropolitan Transportation Commission,6084(157),1


In [63]:
df>>filter(_.project_location.str.contains("San Joaquin County"))>>group_by(_.agency)>>count(_.project_no)

,agency,project_no,n
0,Caltrans,6204(124),1
1,Sacramento County,5924(190),1
2,San Joaquin,5929(243),1
3,San Joaquin Cog,6088(050),1
4,San Joaquin Council Of Governments,6088(051),3
5,San Joaquin Council Of Governments,6088(060),4
6,San Joaquin County,5929(230),1
7,San Joaquin County,5929(243),1
8,San Joaquin County,5929(287),4
9,San Joaquin County,5929(292),2


* `agency` name 'San Joaquin' has same locode as 'San Joaquin County'

In [64]:
df>>filter(_.project_location=="Various Locations (see Comments)")>>count(_.agency, _.project_no)

,agency,project_no,n
0,Imperial,5958(095),1
1,Imperial County,5958(095),3
2,Placer County,5919(119),7
3,Placer County,5919(125),3


#### Checking whats going on in Stanislaus

In [65]:
df>>filter(_.project_location=="Stanislaus County")>>count(_.agency,_.project_no,)

,agency,project_no,n
0,Stan Area Aog,6089(056),1
1,Stan Area Aog,6089(060),1
2,Stan Area Aog,6089(062),1
3,Stanislaus,5938(217),1
4,Stanislaus,5938(232),1
5,Stanislaus Council Of Governments,6089(055),1
6,Stanislaus Council Of Governments,6089(062),1
7,Stanislaus Council Of Governments,6089(079),2
8,Stanislaus County,5938(217),1
9,Stanislaus County,5938(232),2


* multiple `agency` names for Stanislaus County and based on the first four digits of the `project_no` we have only two locodes
    *  Stanislaus County and Stanislaus looks to be the same
   

* despite the 49 entries under Stanislaus, there are no locodes, meaning they are listed as NaN. 
* checking to see if others agencies have NaN

In [66]:
len(df>>filter(_.agency=='Stanislaus'))

49

In [67]:
df>>filter(_.agency=='Stanislaus')>>count(_.locode)

,locode,n


## Non-MPO Agencies
Looking at 'Non-MPO', 'ER NONE' and 'NaN' (or null)

#### Is Null

In [68]:
df>>filter(_.mpo.isnull())

,prefix,project_no,agency,prepared_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,ftip_no,project_location,type_of_work,seq,mpo
20116,HSIP,7504(001),Caltrans,NaN,351200.0,0.0,351200.0,prog code Z300,7504,11,NaN,FTA transfer,Guardrail Var. Locations,1,NaN


### is ER NONE

In [69]:
df>>filter(_.mpo=='ER NONE')>>count(_.agency)

,agency,n
0,California City,1
1,Camarillo,1
2,Chino Hills,1
3,Coachella,1
4,El Centro,1
5,Los Angeles,1
6,Mission Viejo,1
7,Palm Springs,1
8,Pico Rivera,1
9,Ridgecrest,2


In [70]:
df>>filter(_.mpo=='ER NONE')>>filter(_.agency=='Ridgecrest')

,prefix,project_no,agency,prepared_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,ftip_no,project_location,type_of_work,seq,mpo
10180,STP,5385(043),Ridgecrest,2015-01-02,-8826.89,0.0,-82797.55,Authorized,NaN,9,KER050406,N. Downs Street From W. Ward Avenue To W. Inyo...,Pavement Rehabilitation,3,ER NONE
10207,STP,5385(044),Ridgecrest,2014-12-31,-25490.80,0.0,-83405.35,Authorized,NaN,9,KER100405,N. Downs Street From W. Drummond Avenue To W. ...,Pavement Rehabilitation,3,ER NONE


In [71]:
df>>filter(_.mpo=='ER NONE')>>count(_.dist)

,dist,n
0,7,5
1,8,3
2,9,3
3,10,1
4,11,2
5,12,2


In [72]:
df>>filter(_.mpo=='ER NONE')>>filter(_.dist==8)

,prefix,project_no,agency,prepared_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,ftip_no,project_location,type_of_work,seq,mpo
10174,RPMSTP,5282(018),Palm Springs,2014-12-09,0.0,0.0,676628.81,Authorized,NaN,8,NaN,Gene Autry Trail From Whitewater River Wash To...,Widen Bridge And Roadway,9,ER NONE
10176,SRTS,5294(009),Coachella,2015-01-02,0.0,0.0,0.00,Authorized,NaN,8,SCAG016,14 Locations In The Vicinity Of Five Elementar...,"Bulb-outs, Curb Extensions, Flashing Beacons, ...",5,ER NONE
10184,SRTS,5467(007),Chino Hills,2014-10-30,764130.0,0.0,914816.00,Authorized,NaN,8,SCAG016,"Country Club Drive, Williams Avenue, Esther St...","Curb And Gutter, Sidewalk, And Ada-curb Ramps",2,ER NONE


#### Is Non-MPO

In [73]:
df>>filter(_.mpo =='NON-MPO') >> count(_.agency)>>arrange(-_.n)

,agency,n
42,Humboldt County,627
62,Mendocino County,238
72,Nevada County,143
51,Lake County,131
13,Caltrans,105
11,Calaveras County,103
108,Tehama County,85
111,Trinity County,80
80,Plumas County,68
114,Tuolumne County,62
